<!-- TODO: Add logo -->

<div align="center">
    <img src="../assets/dask-logo.svg" alt="Dask logo" width="20%">
</div>

---

# Example: Using Dask, Dask Gateway, and adaptive scaling

In this notebook, we will explore some [Indian stock market data](https://www.kaggle.com/datasets/debashis74017/stock-market-data-nifty-50-stocks-1-min-data?select=ASIANPAINT_minute_data_with_indicators.csv).

## Explore a subset with Dask DataFrame

Let's start with some local (or single-machine) compute.

In the following notebook cells you will read a smaller-subset of the stocks data into a Dask DataFrame and perform some computations.

In [6]:
# Import Dask's Dask DataFrame API
import dask.dataframe as dd

In [9]:
# Read all the CSV files using a glob-pattern into a Dask DataFrame
ddf = dd.read_csv("../<path>/*.csv")

# TODO: Update link with cloud storage link

,date,close,high,low,open,volume,sma5,sma10,sma15,sma20,ema5,ema10,ema15,ema20,upperband,middleband,lowerband,HT_TRENDLINE,KAMA10,KAMA20,KAMA30,SAR,TRIMA5,TRIMA10,TRIMA20,ADX5,ADX10,ADX20,APO,CCI5,CCI10,CCI15,macd510,macd520,macd1020,macd1520,macd1226,MFI,MOM10,MOM15,MOM20,ROC5,ROC10,ROC20,PPO,RSI14,RSI8,slowk,slowd,fastk,fastd,fastksr,fastdsr,ULTOSC,WILLR,ATR,Trange,TYPPRICE,HT_DCPERIOD,BETA
npartitions=50,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
# View the lazy Dask DataFrame
ddf

,date,close,high,low,open,volume,sma5,sma10,sma15,sma20,ema5,ema10,ema15,ema20,upperband,middleband,lowerband,HT_TRENDLINE,KAMA10,KAMA20,KAMA30,SAR,TRIMA5,TRIMA10,TRIMA20,ADX5,ADX10,ADX20,APO,CCI5,CCI10,CCI15,macd510,macd520,macd1020,macd1520,macd1226,MFI,MOM10,MOM15,MOM20,ROC5,ROC10,ROC20,PPO,RSI14,RSI8,slowk,slowd,fastk,fastd,fastksr,fastdsr,ULTOSC,WILLR,ATR,Trange,TYPPRICE,HT_DCPERIOD,BETA
npartitions=50,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
# Inspect the first few values of the DataFrame
ddf.head()

,date,close,high,low,open,volume,sma5,sma10,sma15,sma20,...,fastd,fastksr,fastdsr,ULTOSC,WILLR,ATR,Trange,TYPPRICE,HT_DCPERIOD,BETA
0,2015-02-02 10:18:00+05:30,335.50,335.70,335.40,335.60,1324,335.48,335.500,335.863333,336.3575,...,73.333333,68.836789,83.430256,45.542956,-85.714286,0.597667,0.30,335.533333,28.759935,-1.014028
1,2015-02-02 10:19:00+05:30,335.60,335.65,335.45,335.50,792,335.54,335.490,335.796667,336.2325,...,63.333333,90.375125,80.221964,43.676715,-78.571429,0.569263,0.20,335.566667,29.169152,-1.144243
2,2015-02-02 10:20:00+05:30,335.50,335.60,335.35,335.35,558,335.59,335.525,335.680000,336.1075,...,56.666667,23.658563,60.956826,45.227497,-81.578947,0.546458,0.25,335.483333,29.286752,-0.375410
3,2015-02-02 10:21:00+05:30,335.50,335.55,335.35,335.55,1144,335.54,335.510,335.586667,335.9875,...,56.666667,34.230909,49.421532,52.600565,-73.076923,0.521711,0.20,335.466667,29.051156,-0.499702
4,2015-02-02 10:22:00+05:30,335.45,335.65,335.40,335.50,955,335.51,335.495,335.533333,335.8750,...,42.857143,0.000000,19.296491,57.259216,-62.500000,0.502303,0.25,335.500000,28.655951,0.166799


Calculate the overall highest `high` and lowest `low` in the DataFrame.

In [4]:
ddf.high.max().compute()

4153.0

In [5]:
ddf.low.min().compute()

136.55

As we mentioned earlier, Dask computations look very similar to pandas with an extra `compute()` at the end.

---

## Scale to full dataset with Dask Gateway

You can now scale your computation to all the ~100 files in the dataset using a Dask cluster with Dask Gateway.

### Create a Dask Gateway instance

As the first step, import and instantiate Dask Gateway.

In [ ]:
from dask_gateway import Gateway

gateway = Gateway()
gateway

Open the `Cluster Options` widget where you can view and update cluster configurations like the conda environment, instance type, and any environment variables.

In [ ]:
options = gateway.cluster_options()
options

This is a visual example, but all of this can of course be done programatically:

```python
options.conda_environment = conda_env
options.profile = worker_type
options.environment_vars = {"MYENV": "aNeNvVaR"}
```

> ⚠️ Warning: It's important that the environment used for your notebook (that is, the IPython kernel) must match the Dask worker environment (that is, `options.conda_environment`).

### Create a new Dask cluster and connect to a Client

In [ ]:
# Create a new cluster with the above options
cluster = gateway.new_cluster(options)

In [ ]:
# View the cluster widget
cluster

The cluster starts with zero workers, so you need to set number of workers manually or setup **adaptive scaling**. With adaptive, your cluster can automatically resize itself within the minimum and maximum bounds based on the workload. Learn more in Dask's [adaptive deployments documentation](https://docs.dask.org/en/stable/how-to/adaptive.html).

**In the above UI, set up adaptive with 1 minimum node and 10 maximum nodes.**

Image source: [Dask documentation](https://docs.dask.org/en/stable/how-to/adaptive.html)

<img src="../assets/dask-adaptive.svg" alt="Dask adaptive scaling" width="30%">



In [ ]:
# Enable adaptive scaling
cluster.adapt()

To use adaptive scaling programmatically:
 
```python
cluster.adapt(minimum=10, maximum=100)
```

In [ ]:
# Connect a new client to the Gateway cluster
client = cluster.get_client()

In [ ]:
# View the client widget
client

The `Dask Client` interface gives us a brief summary of everything we've set up so far. 

### Dask's diagnostic dashboard

Open the Dask dashboard by clicking on the link in the Client UI.

Or (recommended), using the lab extension in the left sidebar, open:

* Cluster map
* Task stream
* Worker memory plots
* Progress bar
* Task groups plot

## Computation on the large dataset

In [ ]:
# Read the entire dataset
ddf = dd.read_csv("../<path>/*.csv")

# TODO: Update link with cloud storage link

(Look at the dashboard plots!)

In [ ]:
# Compute highest-high
ddf.high.max().compute()

In [ ]:
# Compute lowest-low
ddf.high.max().compute()

### Standalone example with Dask Array

Copy this snippet to a new cell and execute it!

```python
import dask.array as da

x = da.random.random((100000, 100000), chunks=(1000, 1000))
y = x * x
z = y.mean(axis=1)
z.compute()
```

## Shutdown the cluster

**ALWAYS** remember to shutdown your cluster with the following commands.

> ⚠️ Warning: As with JupyterLab servers, Dask workers run on cloud compute instances and cost actual money.

In [ ]:
cluster.close()
client.close()

---
## 👏 Next:
* [03_managing_environments](../03_managing_environments.ipynb)
---